In [199]:
import argparse
import numpy as np
import random
import argparse
import os
import time
import math
import sys
import json
import csv
import pandas as pd
from itertools import chain

import torch
from torch.autograd import Variable

from sklearn.metrics.pairwise import euclidean_distances, cosine_distances

from models import load_models_decode, generate
from utils import to_gpu, Corpus, batchify, train_ngram_lm, get_ppl

In [200]:
# load_path='output/out1102'
# load_path='output/out_twitter1103'
# load_path='output/out1106_komoran'
# load_path='output/outparsing1204'
load_path='output/output1207_newdata2'
# load_path='output/output1207_newdata_komoran'

In [201]:
idx2word, autoencoder, gan_gen, gan_disc, word2idx= load_models_decode(load_path)

Loading models fromoutput/output1207_newdata2


In [8]:
# vocab2=idx2word
# vocab2
# word2idx

{0: '<pad>',
 1: '<sos>',
 2: '<eos>',
 3: '<oov>',
 4: '가',
 5: '가.',
 6: '가가',
 7: '가거나',
 8: '가게',
 9: '가게가도',
 10: '가게나',
 11: '가게되면',
 12: '가게되었으며',
 13: '가게마다',
 14: '가게에',
 15: '가게에서',
 16: '가게에서나',
 17: '가게주인이주어서.',
 18: '가격',
 19: '가격.',
 20: '가격과',
 21: '가격까지',
 22: '가격대',
 23: '가격대가',
 24: '가격대가있지만',
 25: '가격대도',
 26: '가격대로',
 27: '가격대비',
 28: '가격대에',
 29: '가격대와',
 30: '가격대중성.',
 31: '가격도',
 32: '가격도싸고',
 33: '가격면에서',
 34: '가격보다도',
 35: '가격부담없이',
 36: '가격에',
 37: '가격에다',
 38: '가격에서',
 39: '가격은',
 40: '가격을',
 41: '가격이',
 42: '가격이랑',
 43: '가격이싸고',
 44: '가격저렴.',
 45: '가격저렴하고',
 46: '가격적으로나',
 47: '가격적으로도',
 48: '가격적인',
 49: '가격차이',
 50: '가격치곤',
 51: '가고',
 52: '가공공법',
 53: '가기',
 54: '가까운',
 55: '가까운데서',
 56: '가까운맛.',
 57: '가까움',
 58: '가까움.',
 59: '가까이서',
 60: '가까이에서',
 61: '가깝게',
 62: '가깝다고',
 63: '가끔',
 64: '가끔가다',
 65: '가끔식',
 66: '가끔씩',
 67: '가나',
 68: '가는',
 69: '가는거같다.',
 70: '가는구간에도',
 71: '가는데',
 72: '가는듯한',
 73: '가능.',
 74: '가능하고',
 75: '가능하다',
 76: '가능하다.',
 77: '가능하며

In [252]:
path_raw = 'codedata/DBSCAN_result_Topic_refined2.csv'
maxlen=100

In [148]:
path_raw = 'codedata/1207basic/DBSCAN_result_Topic_refined_Parsed.csv'
maxlen=100

In [253]:
data = pd.read_csv(path_raw, names = None,encoding='cp949')
clusternum=data['clusterno']
docno = data['docNo']
doc = data['raw_doc']

In [236]:
################Komoran 단위
from konlpy.tag import Komoran

pos_tagger = Komoran()

def tokenize_komoran(doc):
    # norm, stem은 optional
    return [x for x, t in pos_tagger.pos(doc)]

token = []
for row in doc :
    token.append(' '.join(k for k in tokenize_komoran(row)))

data['raw_doc']= pd.DataFrame(token)[0]
tokendoc = data['raw_doc']

data

,clusterno,Unnamed: 1,docNo,raw_doc
0,0,0,1024,효과 가 아 있 다
1,0,1,1666,효과 가 아 그다지 ..
2,0,2,1155,효과 있 다
3,0,3,388,효과 가 아 있 어서
4,0,4,2438,효과 가 아 있 어서
5,0,5,449,효과 별로
6,0,6,393,효과 가 아 좋 다
7,0,7,780,효과 가 아 있 으니까
8,0,8,1172,효과 가 아 그리 좋 지 않 아서
9,0,9,1173,효과 가 아 바로 오 아 닿 지 가 아 않 늘 ㄴ다 .


In [150]:
data

,clusterno,Unnamed: 1,docNo,raw_doc
0,-1,0,860_0,하이트는 우선 제일 흔하게 접할 수 있는 맥주 브랜드이다
1,-1,1,9601_1,진함과 부드러움의 조화가 좋아서 향이며 맛이 캔맨주 중 가장 뛰어나다고
2,-1,2,8913_1,한번 들었을때 기억하기 쉬운 이름이다
3,-1,3,9614_4,했던 기억이 나서 슈퍼에서나 마트에서 살때 하이트를 사게 됩니다
4,-1,4,2149_2,맛있게 마신 기억이 있음
5,-1,5,6110_1,저렴한 브랜드 친숙한느낌 하이트에 비해 청량감이 있음
6,-1,6,7041,여자친구가 좋아하더라고요 하이트나 카스나 거의 비슷한데 맛이 깨끗하다나 전 사실 잘...
7,-1,7,10002_1,뭔가 부족한것같은데 그나마 하이트가 맛이 가장 진해서
8,-1,8,6345_2,쉽게 취하지 않아 시원하게 마시기에 좋습니다
9,-1,9,5171_1,먹을때 목넘김이나 개인적으로 좋은 맛을 가지고있음


In [254]:
groupby_cluster = doc.groupby([clusternum])
cluster_doc_rawdata=groupby_cluster.apply(lambda x: x.tolist()).to_dict()

docno_groupby_cluster = docno.groupby([clusternum])
docno_cluster=docno_groupby_cluster.apply(lambda x: x.tolist()).to_dict()


print(len(cluster_doc_rawdata))
if '-1' in cluster_doc_rawdata.keys():
    del cluster_doc_rawdata['-1']  # dbscan 노이즈 결과 버리기
if '-1' in docno_cluster.keys():
    del docno_cluster['-1']  # dbscan 노이즈 결과 버리기
print(len(cluster_doc_rawdata))


#마지막 결과표에 쓰기 위해
li_docno=list(docno_cluster.values())
li_docno = list(chain.from_iterable(li_docno))
len(li_docno)

14
14


684

In [238]:
##########Komoran
groupby_cluster_token = tokendoc.groupby([clusternum])
cluster_doc=groupby_cluster_token.apply(lambda x: x.tolist()).to_dict()
if '-1' in cluster_doc.keys():
    del cluster_doc['-1']

In [181]:
cluster_doc_rawdata

{0: ['큰 효과를 보지는 못함.',
  '효과',
  '효과가 바로 와닿지가 않는다.  \r\n\r\n',
  '효과가 그다지..',
  '효과가 좋다',
  '신속셩 효과',
  '효과없음',
  '효과가 있다',
  '효과가 좋아서',
  '효과가좋다',
  '효과가있다',
  '효과가 신속하게 나서',
  '효과가 있었다',
  '즉시효과',
  '효과를 봤다.',
  '효과가 좋았었다',
  '효과가 좋다',
  '효과좋음',
  '신속한 효과',
  '효과가 신속함',
  '효과가 있으니까',
  '효과가 좋기떄문에',
  '효과가좋다',
  '효과가 있다',
  '큰 효과가 없었음',
  '효과가 좋았음',
  '효과가 좋았다.',
  '효과가 좋다',
  '효과가 그리 좋지 않아서',
  '효과가 딱히 없음',
  '효과적임',
  '효과있어서',
  '효과가 신속함',
  '신속하고 효과적임',
  '효과가 좋아서',
  '효과가 좋아서',
  '효과가 좋아서',
  '효과 별로',
  '효과가 좋아서',
  '효과가 있어서',
  '효과가 좋다',
  '효과가 있음',
  '큰효과는 아직 잘 모르겠ㄸ다',
  '효과가 있어서',
  '크게 효과를 보지 못해서',
  '효과있다',
  '효과가 좋다',
  '신속한 효과',
  '효과가 신속',
  '효과가 좋다',
  '효과가 있어서',
  '효과가 좋음',
  '효과좋음',
  '효과가 별로',
  '효과가 좋음',
  '효과가 별로',
  '효과가 좋음'],
 1: ['효과가 빠르다',
  '사용 편리성과 빠른 효과,효능 때문에',
  '효과가 빠름',
  '편리성 지속성',
  '효과가 빠른 편이다',
  '효과가 빠르다',
  '효과가 빠르고 진정이 잘 되서',
  '효과가 지속되지않아서',
  '사용할 때 효과가 빠르게 나타나서',
  '효과가 빠르다',
  '편하고 효과도 빠름',
  '효과가 바로 나타난다',
  '효과가 빠르다',
  '바로 효과가 나타나서',
  '효과가 빠름

In [239]:
vocab = word2idx
ntokens = len(vocab)

def doc_to_line(cluster_doc, vocab):

    dropped = 0
    linecount = 0
    lines = []
    for line in cluster_doc:
        linecount += 1        
#         if self.lowercase:
#             words = line[:-1].lower().strip().split(" ")
#         else:
#             words = line[:-1].strip().split(" ")
        words = line[:-1].strip().split(" ")
        
        if len(words) > maxlen:
            dropped += 1
            continue
        words = ['<sos>'] + words
        words += ['<eos>']

        # vectorize

        unk_idx = vocab['<oov>']
        indices = [vocab[w] if w in vocab else unk_idx for w in words]

#         print(indices)
        lines.append(indices)
        
    return lines, vocab

In [240]:
def make_code(lines):
    eval_batch_size = 1
    test_data = batchify(lines, eval_batch_size, shuffle=False)
     
    code = []
    for i, batch in enumerate(test_data):
            source, target, lengths = batch
    #         source = to_gpu(True, Variable(source, volatile=True))
    #         target = to_gpu(True, Variable(target, volatile=True))
    #         mask = target.gt(0)
    #         masked_target = target.masked_select(mask)
    #         # examples x ntokens
    #         output_mask = mask.unsqueeze(1).expand(mask.size(0), ntokens)

            # output: batch x seq_len x ntokens
            output = autoencoder.encode(Variable(source), lengths, noise=False)
#             print(output)
            code.append(output)
    return code

In [216]:
title_raw_eucli={}
title_raw_cosine={}

# title_eucli={}
# title_cosine={}

for c in cluster_doc_rawdata.keys() : 
    
########Komoran
# for c in cluster_doc.keys() :     


    lines, vocab = doc_to_line(cluster_doc_rawdata[c], vocab)
    
########Komoran    
#     lines, vocab = doc_to_line(cluster_doc[c], vocab)

    code = make_code(lines)

    code_cat=torch.cat(code,0)

    #code_feature extract(절대값 0.1미만 0으로)
#     code_cat_np = code_cat.data.numpy().squeeze()
#     temp = np.copy(code_cat_np)
#     temp[np.where(np.abs(temp)<feature_range)]=0
#     code_feature=torch.FloatTensor(temp).view(-1,300)

    #code 평균
    code_mean=torch.mean(code_cat,0).view(1,-1)

    #code 평균과 길이가 최소인 vector 구하기
    input1 = code_cat.data.numpy()
    mean_np = code_mean.data.numpy()
    eucli = euclidean_distances(input1, mean_np).squeeze()
    cosine = cosine_distances(input1, mean_np).squeeze()
    
    title_raw_eucli[c]=cluster_doc_rawdata[c][np.argmin(eucli)]
    title_raw_cosine[c]=cluster_doc_rawdata[c][np.argmin(cosine)]
    
    ############Komoran인 경우
#     title_eucli[c]=cluster_doc_rawdata[c][np.argmin(eucli)]
#     title_cosine[c]=cluster_doc_rawdata[c][np.argmin(cosine)]

In [255]:
# title_raw_eucli={}
# title_raw_cosine={}

title_eucli={}
title_cosine={}

# for c in cluster_doc_rawdata.keys() : 
    
########Komoran
for c in cluster_doc.keys() :     


#     lines, vocab = doc_to_line(cluster_doc_rawdata[c], vocab)
    
########Komoran    
    lines, vocab = doc_to_line(cluster_doc[c], vocab)

    code = make_code(lines)

    code_cat=torch.cat(code,0)

    #code_feature extract(절대값 0.1미만 0으로)
#     code_cat_np = code_cat.data.numpy().squeeze()
#     temp = np.copy(code_cat_np)
#     temp[np.where(np.abs(temp)<feature_range)]=0
#     code_feature=torch.FloatTensor(temp).view(-1,300)

    #code 평균
    code_mean=torch.mean(code_cat,0).view(1,-1)

    #code 평균과 길이가 최소인 vector 구하기
    input1 = code_cat.data.numpy()
    mean_np = code_mean.data.numpy()
    eucli = euclidean_distances(input1, mean_np).squeeze()
    cosine = cosine_distances(input1, mean_np).squeeze()
        ############Komoran인 경우
    
    title_eucli[c]=cluster_doc_rawdata[c][np.argmin(eucli)]
    title_cosine[c]=cluster_doc_rawdata[c][np.argmin(cosine)]

In [193]:
title_eucli

{0: '효과가 좋기떄문에',
 1: '효과가 빨라서',
 2: '먹는약보다 효과가 없는것 같다. 뿌리는 기구가 잘 망가진다',
 3: '일시적이라서',
 4: '사용후에 효과가 좋음',
 5: '보통임',
 6: '시원',
 7: '증상이 심할 때만 사용하였고 사용 후 바로 취침에 들어서 정확한 효능을 모른다',
 8: '지속성과 신속성이 있으며, 코가 잘 뚫린다.',
 9: '모름',
 10: '그럭저럭 괜찮았다.',
 11: '효과가 잘나올때도 있고 아닐때도있어서',
 12: '지속력이 오래 간다',
 13: '대체로 무난함',
 14: '코에뿌리면 약품이 흘러 불편함',
 15: '체질',
 16: '제 질환이 만성이라서 약간의 도움만 받았을 뿐입니다.',
 17: '효과있는편',
 18: '순간적',
 19: '주변지인들이 사용해서 써보니 나쁘지 않다 오트리빈과 같은 느낌이라 쓸만하다',
 20: '다른 스프레이 제품과 비교하여 좋은 점이 없음',
 21: '증세완화뿐 사용하지않으면 다시 증상생김',
 22: '코액 스프레이헤두 뻥뚤리는 느낌이 없. 코가 아플때두 있구...',
 23: '코에 자극이 강한느낌이 들어서',
 24: '비염이 심한정도여서 큰효과를 기대하진 않았고 그렇게 느꼈다',
 25: '예전에 사용해봤을 때는 쓰다는 느낌만 강함',
 26: '의사가 유명한거라고 추천해줘서 지금까지 쓰고있고 잘 듣는편이다',
 27: '코 막힐때 뚫어줬었지만 자주 목으로 넘어갔었음',
 28: '특별히 드라마틱한 효과를 보지 못해서',
 29: '코가 막혀서 잠자기 불편할때 코가 뚤려서 편함 하지만 목으로 액이 넘어서와서 불편함으로 만족하는편',
 30: '이유없음',
 31: '진료 당시 의사가 처방해 주었으며, 처방 해준 스프레이가 효과가 있었기 때문에 추후에도 계속 처방을 요구 했음',
 32: '효과는 나쁘지 않았으나 증상이 자주 나타나 많이 써야했기 때문에',
 33: '뿌리면 목이 아픔',
 34: '효과와 지속성 둘 다 그닥 나쁜편이 아니

In [189]:
title_raw_cosine

{0: '효과가 그다지..',
 1: '편하고 효과도 빠름',
 2: '생각보다 나아지는걸 못 느꼇다.',
 3: '큰 효과를 못봤고 일시적이다.',
 4: '휴대성이 좋고 효과도 좋았다',
 5: '신속한 효능',
 6: '코막힌것이 잘뚫린 느낌이고 시원해서',
 7: '뿌리고 자니까 증상이 완화되는 느낌이었음',
 8: '생각보다 효과가 오래지속되지 않았다',
 9: '눈이 띄는 효과는 모르겠다',
 10: '효과가 어느정도 있어서',
 11: '사용했을때 일시적이라도 코막힘이 완화되는 현상 때문에',
 12: '잠깐 시원하게뚫리는거 같지만 지속력은 없는거같다',
 13: '나름 효과가 있기때문에',
 14: '코에 뿌리고 난후 효과가 괜찮아서 사용하겠됐음.',
 15: '약효가 오래가고 부작용이 적은거같다',
 16: '효과가 즉효성이고 나한테 제일 잘맞아서',
 17: '효과가 좋은 편이다.',
 18: '뿌리기 전보다 숨쉬기 편해짐',
 19: '주변 지인 추천으로 사용하였는데 효과가 괜찮음.',
 20: '다른제품과 특별히 다른 점을 찾지 못했다.',
 21: '내성이 생기는 듯한 생각이 듭니다.',
 22: '코가뚤리는 느낌이 시웒하다',
 23: '코에 약간 자극이 있는것 같아서',
 24: '비염이 심한정도여서 큰효과를 기대하진 않았고 그렇게 느꼈다',
 25: '부작용없이 무난하게 사용중이며 빠르지는 않지만 증상이 완화되어서 만족한다.',
 26: '효과가 좋았으나 사용할 양이 부족',
 27: '코가 약간 조이는 느낌이 든다고 함.',
 28: '드라마틱한 효과는 없기 때문이다.',
 29: '사용하면 편안한 느낌이 한번씩 드는것같습니다',
 30: '혁신적인 제품이다',
 31: '그냥 의사가 처방해 줘서 사용해봄',
 32: '효과가 신속하게 나타나고, 지속력이 길어서 사용하기 편리하다',
 33: '뿌리면 목이 아픔',
 34: '효과는 잇는것도 같은데 그닥 신박하다는 느낌은 없어서',
 35: '가지고 다니기 편하고 즉각적인 효과가 

In [220]:
df = pd.DataFrame([title_raw_eucli,cluster_doc_rawdata]).T
df['clusterNo'] = df.index
cosine_title=list(title_raw_cosine.values())
cosine_title
df[2]=cosine_title

In [256]:
########Komoran
df = pd.DataFrame([title_eucli,cluster_doc]).T
df['clusterNo'] = df.index
cosine_title=list(title_cosine.values())
cosine_title
df[2]=cosine_title

In [257]:
df

,0,1,clusterNo,2
0,효과가 있다,"[효과 가 아 있 다, 효과 가 아 그다지 .., 효과 있 다, 효과 가 아 있 어...",0,효과가 있다
1,효과가 지속적이지 않다,"[효과 가 아 오래 지속 되 지 않 아서, 지속 성, 효과 가 아 지속 되 지 않 ...",1,효과가 지속적이지 않다
2,효과가 바로 나타난다,"[효과 가 아 금방 나타나 아 ㄴ다, 효과 가 아 바로 나타나 아 ㄴ다, 효과 가 ...",2,효과가 바로 나타난다
3,잘모름,"[잘 모르 ㅁ, 잘 모르 하 겠 음, 모르 ㅁ, 모르 아서, 잘 모르 하 겠 다]",3,잘모름
4,보통이라서,"[보통 이 라 서, 보통 이 ㅁ, 보통 이 이 니까, 보통 이 다, 보통 이 라 서...",4,보통이라서
5,효과가 빠르다,"[효과 가 아 빠 르 게 오 오 았 다, 효과 가 아 빠 르 다, 빠 르 ㄴ 효과 ...",5,효과가 빠르다
6,시원하다,"[시원 하 다 하 아서, 시원, 시원 하 다, 시원 하 다 . 잘 듣 늘 ㄴ다 .,...",6,별로 시원한 효과가 없음
7,일시적인 효과,"[일시 적 이 지만 효과 가 아 있 다 이 고 하 니 . ., 일시 적 효과 가 아...",7,일시적인 효과
8,효과가 좋은 편이다.,"[효과 가 아 좋 은 편 이 다 ., 효과 있 늘 ㄴ 편, 잘 듣 늘 ㄴ 편 이 다...",8,효과가 좋은 편이다.
9,사용이 편리해서. 효과가 좋아서,"[효과 가 아 신속 하 이 며 , 사용 이 편리 하 ㅁ, 편리 하 고 효과 좋 다,...",9,사용이 편리해서. 효과가 좋아서


In [258]:
rows = []
_ = df.apply(lambda row: [rows.append([row['clusterNo'],nn,row[0],row[2]]) 
                         for nn in row[1]], axis=1)

In [247]:
rows

[[0, '효과 가 아 있 다', '효과 가 있 다', '효과 가 있 다'],
 [0, '효과 가 아 그다지 ..', '효과 가 있 다', '효과 가 있 다'],
 [0, '효과 있 다', '효과 가 있 다', '효과 가 있 다'],
 [0, '효과 가 아 있 어서', '효과 가 있 다', '효과 가 있 다'],
 [0, '효과 가 아 있 어서', '효과 가 있 다', '효과 가 있 다'],
 [0, '효과 별로', '효과 가 있 다', '효과 가 있 다'],
 [0, '효과 가 아 좋 다', '효과 가 있 다', '효과 가 있 다'],
 [0, '효과 가 아 있 으니까', '효과 가 있 다', '효과 가 있 다'],
 [0, '효과 가 아 그리 좋 지 않 아서', '효과 가 있 다', '효과 가 있 다'],
 [0, '효과 가 아 바로 오 아 닿 지 가 아 않 늘 ㄴ다 .', '효과 가 있 다', '효과 가 있 다'],
 [0, '효과 가 아 좋 다', '효과 가 있 다', '효과 가 있 다'],
 [0, '효과 가 아 있 다', '효과 가 있 다', '효과 가 있 다'],
 [0, '효과 가 아 좋 아서', '효과 가 있 다', '효과 가 있 다'],
 [0, '효과 가 아 좋 음', '효과 가 있 다', '효과 가 있 다'],
 [0, '효과 가 아 별로', '효과 가 있 다', '효과 가 있 다'],
 [0, '효과 가 아 좋 아서', '효과 가 있 다', '효과 가 있 다'],
 [0, '크 ㄴ 효과 늘 ㄴ 아직 잘 모르겠ㄸ다', '효과 가 있 다', '효과 가 있 다'],
 [0, '효과 가 아 좋 음', '효과 가 있 다', '효과 가 있 다'],
 [0, '효과 가 아 좋 다', '효과 가 있 다', '효과 가 있 다'],
 [0, '효과 가 아 좋 다', '효과 가 있 다', '효과 가 있 다'],
 [0, '효과 가 아 좋 아서', '효과 가 있 다', '효과 가 있 다'],
 [0, '효과 를 보 오 았 다 .', '효과 가 있 다', '효

In [259]:
df_new = pd.DataFrame(rows, columns=['clusterNo','raw_docs','title_euclidean','title_cosine'])
df_new['docno']=li_docno
df_new

,clusterNo,raw_docs,title_euclidean,title_cosine,docno
0,0,효과 가 아 있 다,효과가 있다,효과가 있다,1024
1,0,효과 가 아 그다지 ..,효과가 있다,효과가 있다,1666
2,0,효과 있 다,효과가 있다,효과가 있다,1155
3,0,효과 가 아 있 어서,효과가 있다,효과가 있다,388
4,0,효과 가 아 있 어서,효과가 있다,효과가 있다,2438
5,0,효과 별로,효과가 있다,효과가 있다,449
6,0,효과 가 아 좋 다,효과가 있다,효과가 있다,393
7,0,효과 가 아 있 으니까,효과가 있다,효과가 있다,780
8,0,효과 가 아 그리 좋 지 않 아서,효과가 있다,효과가 있다,1172
9,0,효과 가 아 바로 오 아 닿 지 가 아 않 늘 ㄴ다 .,효과가 있다,효과가 있다,1173


In [261]:
df_new.to_csv('1207_med_komoran_center_vector_title_400.csv',sep=',',encoding='CP949')

In [165]:
parseno=df_new['docno']
title=df_new['title_cosine']
myzip = zip(parseno, title)
result_doc = dict(myzip)
len(result_doc)

13910

In [166]:
parseno=df_new['docno']
cluster=df_new['clusterNo']
myzip = zip(parseno, cluster)
number = dict(myzip)
len(number)

13910

In [29]:
result_doc

{'6980_1': '느낌이 좋다',
 '8990': '느낌이 좋다',
 '1606_2': '느낌이 좋다',
 '10239': '느낌이 좋다',
 '2514': '느낌이 좋다',
 '1774': '느낌이 좋다',
 '5684_5': '느낌이 좋다',
 '124_2': '느낌이 좋다',
 '8015': '느낌이 좋다',
 '8360_1': '느낌이 좋다',
 '6452': '느낌이 좋다',
 '4776': '느낌이 좋다',
 '5150': '느낌이 좋다',
 '8019_0': '느낌이 좋다',
 '1239_1': '느낌이 좋다',
 '1024_1': '느낌이 좋다',
 '7341': '느낌이 좋다',
 '9318_2': '느낌이 좋다',
 '3898_1': '느낌이 좋다',
 '5586': '느낌이 좋다',
 '3957_1': '느낌이 좋다',
 '6919_1': '느낌이 좋다',
 '3838_1': '느낌이 좋다',
 '8040_2': '느낌이 좋다',
 '4453': '느낌이 좋다',
 '6857': '느낌이 좋다',
 '818_1': '느낌이 좋다',
 '4747_2': '느낌이 좋다',
 '437_1': '느낌이 좋다',
 '2195': '느낌이 좋다',
 '2777_0': '느낌이 좋다',
 '8734_2': '느낌이 좋다',
 '7886_1': '느낌이 좋다',
 '4167_3': '느낌이 좋다',
 '9841_1': '느낌이 좋다',
 '485_0': '느낌이 좋다',
 '2_1': '느낌이 좋다',
 '1566': '느낌이 좋다',
 '7169_1': '느낌이 좋다',
 '7896': '느낌이 좋다',
 '8311_1': '느낌이 좋다',
 '7865_1': '느낌이 좋다',
 '2795': '느낌이 좋다',
 '328': '느낌이 좋다',
 '720_1': '느낌이 좋다',
 '5384': '느낌이 좋다',
 '6061_0': '느낌이 좋다',
 '6384_1': '느낌이 좋다',
 '6861': '느낌이 좋다',
 '1886': '느낌이 좋다'

# 마지막 결과파일

In [167]:
path_raw = 'rawData.csv'
maxlen=100

In [168]:
data = pd.read_csv(path_raw, names = None,encoding='cp949')
docno = data['no']
doc = data['응답값']

In [169]:
myzip = zip(docno, doc)
raw_docs = dict(myzip)
len(raw_docs)

10400

In [141]:
raw_docs

{2: '씁쓸한 맛이 좋다. 톡쏘는 느낌이 좋다.',
 3: '시원함',
 4: '칼로리가 낮아서 많은 양을 먹어도 무리가 없음    맥주를 많이 마시는 편이기 때문에 배가 안 부른 맥주가 좋음',
 5: '칼로리 걱정을 조금 덜 할 수 있고 가볍게 마시기 참 좋다',
 6: '레몬맛이 좋아서',
 7: '내가 좋아하는 맛이다. 맥주의 고소한 맛과 탄산이 적절하게 들어가있다.',
 8: '가벼운맛',
 9: '맛이 깔끔함',
 10: '다른 맥주와 차별화 된 맛.',
 11: '브랜드 이미지, 즉 젊은 남자의 이미지를 가졌으며 맛또한 뛰어나다 특히 깔끔한 뒷맛이 좋다',
 12: '맥주거품맛이 부드럽다',
 13: '부드러워서',
 14: '맛있어요',
 15: '목넘김이 부드러워서',
 16: '맛이있어서',
 17: '톡쏘는 맛이 좋아서',
 18: '맛을 좋아해서 그 상표를 찾는다. 페트병맥주중에서는 선호 하는 맛이다.',
 19: '깔끔하다',
 20: '일반 생맥주나 캔맥주보다 맥주맛이 덜나고 순해서 마시기 편하다',
 21: '심플하다',
 22: '디자인이 이쁨',
 23: '맛있어서',
 24: '맛이 독특하고 맛있다',
 25: '흑맥주중에서도 저렴하고    맛이 덜씀',
 26: '거품이 부드럽고 목넘김이 좋다. 향또한 마찬가지',
 27: '맛있어서.  ',
 28: '쌉싸릅한 흑맥주  부드러운 거품  ',
 29: '레몬맛이 맛있음',
 30: '부드러운 목넘김',
 31: '맛이 부드럽고 끝맛이 좋음',
 32: '먹기 부담스럽지 않고 쉽게 먹을 수 있다',
 33: '시원한느낌이 좋아요.',
 34: '맛잇는 맥주라서 카스를 먹게된다.',
 35: '거품 엔젤링때문에',
 36: '맥주의 맛이 좋아서',
 37: '부드러운 목넘김때문에',
 38: '맛있어요',
 39: '선전에 공유가 나와서',
 40: '맛있기 때문에',
 41: '맥주가 부드럽고 쓴맛이 거의 없다',
 42: '똑쏘는 맛',
 43: '맛있다',
 44: '특유의

In [18]:
# final={}
# for key in raw_docs.keys():
#     coding=[]
#     for docno in result_doc.keys():
#         if str(key)==str(docno.split('_')[0]):
#             coding.append(result_doc[docno])
#     final[key]=coding

In [ ]:
# finallist=[]
# for key in raw_docs.keys():
#     row=[]
#     row.append(key)
#     row.append(raw_docs[key])
#     coding=[]
#     for docno in result_doc.keys():
#         if str(key)==str(docno.split('_')[0]):
#             coding.append(result_doc[docno])
#     row.extend(coding)   
#     finallist.append(row)

In [170]:
finallist=[]
for key in raw_docs.keys():
    row=[]
    row.append(key)
    row.append(raw_docs[key])
    coding=[]
    for docno in number.keys():
        if str(key)==str(docno.split('_')[0]):
            coding.append(number[docno])
            coding.append(result_doc[docno])
    row.extend(coding)   
    finallist.append(row)

In [171]:
print(finallist)

[[2, '씁쓸한 맛이 좋다. 톡쏘는 느낌이 좋다.', '0', '느낌이 좋다', '113', '톡쏘는 맛이 일품이다'], [3, '시원함', '16', '시원하고'], [4, '칼로리가 낮아서 많은 양을 먹어도 무리가 없음    맥주를 많이 마시는 편이기 때문에 배가 안 부른 맥주가 좋음', '121', '레몬맛이 상쾌함을 줘서'], [5, '칼로리 걱정을 조금 덜 할 수 있고 가볍게 마시기 참 좋다', '148', '과일향에 간단히 부담없이 마실 수 있어서', '91', '친구들과 마시기 좋다'], [6, '레몬맛이 좋아서', '121', '레몬맛이 상쾌함을 줘서'], [7, '내가 좋아하는 맛이다. 맥주의 고소한 맛과 탄산이 적절하게 들어가있다.', '244', '마셔도 배가많이 안불러서 알콜도수가 낮아 순하다', '36', '맛이 내가좋아하는맛이다'], [8, '가벼운맛', '1', '가볍고'], [9, '맛이 깔끔함', '2', '맛이 깔끔하고'], [10, '다른 맥주와 차별화 된 맛.', '124', '가장 보편적으로 접할수 있고'], [11, '브랜드 이미지, 즉 젊은 남자의 이미지를 가졌으며 맛또한 뛰어나다 특히 깔끔한 뒷맛이 좋다', '3', '뒷맛이 깔끔하다'], [12, '맥주거품맛이 부드럽다', '146', '거품과 맛이 좋다'], [13, '부드러워서', '4', '부드럽고'], [14, '맛있어요', '5', '맛있다'], [15, '목넘김이 부드러워서', '6', '목넘김이 부드럽고'], [16, '맛이있어서', '18', '맛이 순함'], [17, '톡쏘는 맛이 좋아서', '18', '맛이 순함'], [18, '맛을 좋아해서 그 상표를 찾는다. 페트병맥주중에서는 선호 하는 맛이다.', '274', '최근 맥주중에 가장 깔끔한 맛이 나서 선호합니다', '36', '맛이 내가좋아하는맛이다'], [19, '깔끔하다', '7', '깔끔하고'], [20, '일반 생맥주나 캔맥주보다 맥주맛이 덜나고 순해서 마시기 편하다', '139', '캔

In [172]:
with open('output1207_re.csv', 'w', newline='', encoding='CP949') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerows(finallist)